This notebook creates a dictionary of $(n , \gamma)$ cross sections, $\sigma$ 


The first two cells well install the python libraries astropy and wnutils, with the proper version of python that is associated with the users notebook. If the user already has these libraries installed, the first two cells need not be run.

In [1]:
import sys
!{sys.executable} -m pip install --quiet wnutils

In [2]:
import sys
!{sys.executable} -m pip install --quiet astropy

In [3]:
import os, io, requests
from lxml import etree

We now import the necessary libraries needed to calculate and plot our cross sections $\sigma$ as a function of neutron number $N$, or atomic mass $A$.

In [4]:
import wnutils.xml as wx
import numpy as np
from astropy import constants as pc
from astropy import units as u
from matplotlib import pyplot as plt
import itertools as it

We now import an xml file

In [5]:
my_xml = wx.Xml(io.BytesIO(requests.get('https://osf.io/vqhxr/download').content))

We need to define a few variables

In [6]:
t9 = 0.8
T = t9 * 1.e9 * u.K
Z = str(43)

Now we calculate the thermal velocity $v_t$

In [7]:
v_t = np.sqrt((2 * pc.k_B.decompose(u.cgs.bases) * T) / pc.m_n.decompose(u.cgs.bases))

We need a few dictionaries

$\underline{sig\_dict}$ is the one we are interested, while $\underline{filter}$ is only used to cross reference with $\underline{nuclides}$

In [8]:
sig_dict = {}

filter = {}

In [9]:
nuclides = {}
nuc = my_xml.get_nuclide_data("[z = " + Z +"]")
nuclides.update(nuc)

We obtain our reaction data for neutron captures $(n,\gamma)$ using wnutils

In [10]:
reac = my_xml.get_reaction_data("[reactant = 'n' and count(reactant) = 2  and product = 'gamma' and count(product) = 2]")

Now loop over all reactions to obtain the rates and convert them to cross sections $\sigma$. We then convert to millibarns $mb$ and strip the units since python in general doesnt work well with units. Then populate the dictionary

In [11]:
for s in reac:
    reaction = reac[s]
    rate = reaction.compute_rate(t9) * u.cm**3 /u.second /u.mol 
    sigma = (rate / (pc.N_A * v_t)).to(u.barn)
        
    usigma = 1000 * sigma / u.barn
    sig_dict[s] = usigma        

We use the filter dictionary to match nuclides and reac

In [12]:
for reaction in reac:
    for reactant in reac[reaction].reactants:
        if reactant in nuclides:
            filter[reaction] = reaction

Now if the key is in $\underline{sig\_dict}$ but not in $\underline{filter}$ we delete it from $\underline{sig\_dict}$ to get a dictionary of the cross sections we are interested in.

In [13]:
for key in list(sig_dict.keys()):
    if key not in filter:
        del sig_dict[key]

In [14]:
print('Cross Sections at T9 of ' + str(t9))

for k,v in sorted(sig_dict.items()):
    print(str(k)+': ' + str(v) + ' mb')

Cross Sections at T9 of 0.8
n + tc100 -> tc101 + gamma: 682.9014105179624 mb
n + tc101 -> tc102 + gamma: 171.89596373861357 mb
n + tc102 -> tc103 + gamma: 475.5629597706425 mb
n + tc103 -> tc104 + gamma: 140.10032671640542 mb
n + tc104 -> tc105 + gamma: 553.9022965260385 mb
n + tc105 -> tc106 + gamma: 71.99278573279153 mb
n + tc106 -> tc107 + gamma: 345.7871529023834 mb
n + tc107 -> tc108 + gamma: 78.9492252244812 mb
n + tc108 -> tc109 + gamma: 415.79436172877763 mb
n + tc109 -> tc110 + gamma: 88.38880231661076 mb
n + tc110 -> tc111 + gamma: 204.12059193577988 mb
n + tc111 -> tc112 + gamma: 44.428546227221574 mb
n + tc112 -> tc113 + gamma: 86.27031833144166 mb
n + tc113 -> tc114 + gamma: 12.364733418799595 mb
n + tc114 -> tc115 + gamma: 58.222037821514775 mb
n + tc115 -> tc116 + gamma: 8.335463053088931 mb
n + tc116 -> tc117 + gamma: 27.256580083741504 mb
n + tc117 -> tc118 + gamma: 5.409387545122122 mb
n + tc118 -> tc119 + gamma: 26.356062128409278 mb
n + tc119 -> tc120 + gamma: 2.945